In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-txt/test0730.txt


In [2]:
!pip install peft==0.4.0
!pip install transformers==4.30.2
!pip install datasets==2.12.0
!pip install tqdm==4.65.0
!pip install loguru==0.7.0
!pip install fire==0.5.0
!pip install bitsandbytes==0.39.0
#wandb==0.15.3
!pip install cpm_kernels==1.0.11
!pip install accelerate==0.20.3
!pip install sentencepiece==0.1.99
!pip install deepspeed==0.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=f5aba8e123e4a43d4bb3428c896e73e2fab8f26adac52e66a7795a9fe971021c
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 12.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 9.9 MB/s eta 0:00:0000:01
     ━━━━━

In [5]:
import os
import argparse
from typing import List, Dict, Optional
from accelerate import init_empty_weights  # load an empty model,just structure , no real weight.
import bitsandbytes as bnb
import torch
from glob import glob
from loguru import logger
from datasets import load_dataset
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    set_seed,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig 
)
from peft import (
    TaskType,
    LoraConfig,
    #AdaLoraConfig ,  #  提出自2020年 感觉和lora区别不大 而且和qlora有冲突 这里代码没有用到 
                     #例子https://www.zhihu.com/question/596950521/answer/3109759716
    get_peft_model,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training
)
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
from transformers.deepspeed import HfDeepSpeedConfig
import deepspeed
import json
from itertools import chain

In [6]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [7]:
string_list ="我<unk></s>"
#string_list ="11+2+3+4"


tokenized_res = tokenizer.encode(string_list,add_special_tokens=True,padding=True)
logger.info(tokenized_res)


2023-07-31 11:01:08.230 | INFO     | __main__:<module>:6 - [64790, 64792, 34211, 31002, 3013, 6029, 30917, 30994]


数字开头的话 前面会加上三个 "" 空白
而非数字开头 前面加2个空白 怪的很

In [8]:
tokenizer.batch_decode(tokenized_res,add_special_tokens=True)

['', '', '我', '<', 'unk', '></', 's', '>']

tokenizer.pad_token_id =0  pad_token=<unk>
    eos_token_id=2 eos_token=</s>

In [9]:
logger.info(f"tokenizer.pad_token_id={tokenizer.pad_token_id},tokenizer.pad_token={tokenizer.pad_token}")

2023-07-31 11:01:16.541 | INFO     | __main__:<module>:1 - tokenizer.pad_token_id=0,tokenizer.pad_token=<unk>


In [10]:
logger.info(f"tokenizer.pad_token_id={tokenizer.eos_token_id},tokenizer.pad_token={tokenizer.eos_token}")

2023-07-31 11:01:19.595 | INFO     | __main__:<module>:1 - tokenizer.pad_token_id=2,tokenizer.pad_token=</s>


In [11]:
logger.info(f"tokenizer.pad_token_id={tokenizer.bos_token_id},tokenizer.pad_token={tokenizer.bos_token}")

Using bos_token, but it is not set yet.
2023-07-31 11:01:22.910 | INFO     | __main__:<module>:1 - tokenizer.pad_token_id=None,tokenizer.pad_token=None


In [114]:
def get_dataset(data_path, tokenizer, global_args=None,max_samples=None):
    """读取本地包含json/jsonl文件的目录，将目录中所有文件作为dataset，并tokenize，shuffle，返回datasets.dataset"""
    
    if not (data_path is not None and os.path.exists(data_path)):
        raise ValueError("data_path requires a directory pointing to  .txt files")
    """https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py#L383"""
    data_files_list = glob(f'{data_path}/**/*.txt', recursive=True) 
    logger.info(f"data files: {', '.join(data_files_list)}")
          
    extension = "text"
    # sasmple_by="document" 就是整个txt全部读取了
    raw_datasets = load_dataset(
        extension,
        data_files=data_files_list,
        sample_by="document"
    )
#     logger.info(f"在取样之前 data len ={len(data['train'])}")
#     if max_samples is not None and max_samples > 0:
#             max_samples = min(len(data['train']), max_samples)  # 
#             data['train'] =  data['train'].select(range(max_samples))
#     logger.info(f"在取样之后 data len ={len(data['train'])}")
    
#     column_names = data['train'].column_names  # remove_columns=column_names  ,remove all at once
#     """tokenize_func 中是单样本处理的写法 所以这里的batched只能设置为False"""
#     logger.info("preprocessing dataset...")
#     dataset = data['train'].map(lambda example: tokenize_func(example, tokenizer, global_args),
#                                 batched=False, 
#                                 remove_columns=column_names)
#     dataset = dataset.shuffle(seed=global_args.seed)
#     dataset = dataset.flatten_indices()
    return raw_datasets

In [115]:
def tokenize_function(examples,tokenizer):
    # 这个函数必须返回dict 
    ## 不然raw_datasets.map(tokenize_function, ...)调用会提示报错
    ## 要么就直接这么写  return tokenizer(examples["text"])
    return {"input_ids" :tokenizer(examples["text"]).input_ids}

In [116]:
block_size = 10

In [117]:
def group_texts(examples,block_size):
        # Concatenate all texts.
        #concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        total_length = len(examples[list(examples.keys())[0]])
        # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
        print(f"total_length={total_length}")
        
        #ADD 20230731 原先shiming是去掉了最后的尾巴 但是我觉得可以保留 毕竟语料不多 每一部分都要用上
        #if total_length >= block_size:
        #    total_length = (total_length // block_size) * block_size
        
        # Split by chunks of max_len.
        result = {
            k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
            for k, t in examples.items()
        }
        #result["labels"] = result["input_ids"].copy()
        return result

In [118]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

In [119]:
pad_token_id = tokenizer.pad_token_id
print(f"pad_token_id = {pad_token_id}")

pad_token_id = 0


In [121]:
import random
def get_chained_lm_datasets(lm_datasets,
                            pad_token_id: int ,
                            global_args_max_length: int = 2048,
                            num_samples=-1,
                            tokenizer=None):
    "做padding和根据global.max_length截取"
    input_ids_list = list(chain(*lm_datasets['input_ids']))
    # padding  with max_len_of_samples
    max_len_ids = max([len(input_ids) for input_ids in input_ids_list] ) 
    print(f"样本中ids 最大长度 max_len_ids={max_len_ids}")
    print(f"全局训练参数的global_args_max_length={global_args_max_length}")
    print(f"cutoff to len = {min(global_args_max_length,max_len_ids)}")
    
    #截断
    new_input_ids=[]
    for ids in input_ids_list :
        print(f"len_sample_ids={len(ids)}")
        pad_len = max_len_ids - len(ids)
        ids += [pad_token_id]* pad_len
        # 截取 根据命令行输入的max len和 ids_list中最大长度 两值做比较取较小的来截取前面N个
        ids =ids[: min(global_args_max_length,max_len_ids)]
        new_input_ids.append(ids)
    # 抽样
    random_chosen_samples=new_input_ids
    print(f"验证查看 shuffle之前 第一个样本\n{tokenizer.decode(random_chosen_samples[0])}")
    print(f"验证查看 shuffle之前 最后一个样本\n{tokenizer.decode(random_chosen_samples[-1])}")
    random.shuffle(new_input_ids)
    if num_samples > -1: # num_samples  一般只是debug的时候取少量样本测试 ；
        #random.sample 不重复抽样
        random_chosen_samples = random.sample(new_input_ids,k=num_samples)
    print(f"样本数量={len(random_chosen_samples)}")
    #return {"input_ids":random_chosen_samples  ,"labels":random_chosen_samples.copy()}
    return [{"input_ids": item ,"labels":item.copy() } for item in random_chosen_samples]

In [92]:
def fault_tolerance_data_collator(features: List) -> Dict[str, Any]:
    if not isinstance(features[0], Mapping):
        features = [vars(f) for f in features]
    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if type(first["label_ids"][0]) is int else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    try:
        for k, v in first.items():
            if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
                if isinstance(v, torch.Tensor):
                    batch[k] = torch.stack([f[k] for f in features])
                elif isinstance(v, np.ndarray):
                    batch[k] = torch.tensor(np.stack([f[k] for f in features]))
                else:
                    batch[k] = torch.tensor([f[k] for f in features])
    except ValueError:  # quick fix by simply take the first example
        for k, v in first.items():
            if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
                if isinstance(v, torch.Tensor):
                    batch[k] = torch.stack([features[0][k]] * len(features))
                elif isinstance(v, np.ndarray):
                    batch[k] = torch.tensor(np.stack([features[0][k]] * len(features)))
                else:
                    batch[k] = torch.tensor([features[0][k]] * len(features))
    #print("打印fault_tolerance_data_collator处理后的信息")
    #print(f"batch.keys()={batch.keys()},len_batch_key_labels={len(batch['labels'])}    ,len_batch_key_k={len(batch[k])}")
    return batch

In [93]:
a=[1,2,3]
random.sample(a,k=2)

[3, 2]

In [122]:
# 是两段文本
raw_datasets = get_dataset(data_path="/kaggle/working/test",tokenizer=tokenizer,max_samples=10000)
print(raw_datasets)
print(raw_datasets['train']['text'])

2023-07-31 14:05:20.255 | INFO     | __main__:get_dataset:8 - data files: /kaggle/working/test/test/test0730.txt, /kaggle/working/test/test/test0730_2.txt


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2
    })
})
["1+2+3+466666 66666 hgsg sgs hshshs hsfh shsh sh dh fhh\n4+3+2=1                     afghu wfsvjoer n gw jgohuth4odjf ajaddpjfwon sdjnfslf  \n我很优秀  你呢？我不知道啊，哈哈。。。 vsoro nvlknlbh h rwlj flvnskd gb gd \n吃放睡觉打豆豆哦，【】[];'lvsvd a  gg  \\n brh5p bph owgwk rwf s59jbg\nhd g  ghr5 eg r hsghrwgrwhhwh hg  wfsvjoer\ng shth h j efwe gb\n\nef g h hsr \n\n\nge whreh \n hgs ghhth", "1+2+3+466666 66666 hgsg sgs hshshs hsfh shsh sh dh fhh\n4+3+2=1                     afghu wfsvjoer n gw jgohuth4odjf ajaddpjfwon sdjnfslf  \n我很优秀  你呢？我不知道啊，哈哈。。。 vsoro nvlknlbh h rwlj flvnskd gb gd \n吃放睡觉打豆豆哦，【】[];'lvsvd a  gg  \\n brh5p bph owgwk rwf s59jbg\nhd g  ghr5 eg r hsghrwgrwhhwh hg  wfsvjoer\ng shth h j efwe gb\n\nef g h hsr \n\n\nge whreh \n hgs ghhth"]


In [124]:
tokenized_datasets = raw_datasets.map(
                lambda examples:tokenize_function(examples,tokenizer),
                batched=True,
                num_proc=2,#data_args.preprocessing_num_workers,
                remove_columns=['text'],#column_names,
                #load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on dataset",
            )
print(tokenized_datasets)
print(tokenized_datasets['train']['input_ids'])

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 2
    })
})
[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972, 30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978, 30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917, 299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578, 13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296, 10789, 30927, 11408, 271, 9563, 30933, 5282, 266, 310, 317, 30929, 467, 30927, 1063, 1056, 30972, 374, 30965, 30926, 29953, 2065, 30925, 30965, 30926, 23347, 268, 30921, 30965, 30916, 30926, 5239, 30926, 265, 13, 36545, 31994, 265, 54622, 55282, 31514, 42660, 55674, 31123, 34380, 39507, 5088, 14724, 310, 30933, 30920, 16462, 26604, 30919, 299, 416, 30929, 30920, 30965, 795, 30933, 30916, 2823, 30921, 317, 30931, 317, 30921, 30910, 13, 55058, 54804, 35228, 54819, 55720, 55720, 56645, 31123, 31538, 31529, 30995, 4571, 30953, 30920, 30933, 13532, 30921, 260, 265, 1298, 265, 

In [243]:
#raw_data['train']['text']

In [128]:
block_size=50

In [129]:
lm_datasets = tokenized_datasets['train'].map(
                lambda examples :group_texts(examples ,block_size) ,
                batched=False,
                num_proc=2,#data_args.preprocessing_num_workers,
                #load_from_cache_file=not data_args.overwrite_cache,
                desc=f"Grouping texts in chunks of {block_size}",
                #remove_columns =['attention_mask', 'position_ids',]
            )
print(lm_datasets)

Grouping texts in chunks of 50 (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

total_length=219total_length=219

Dataset({
    features: ['input_ids'],
    num_rows: 2
})


In [130]:
dataset_train = get_chained_lm_datasets(lm_datasets,
                        pad_token_id=tokenizer.pad_token_id,
                        global_args_max_length=50000,
                         num_samples=-1,
                        tokenizer = tokenizer)
print(dataset_train)

样本中ids 最大长度 max_len_ids=50
全局训练参数的global_args_max_length=50000
cutoff to len = 50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=19
len_sample_ids=50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=19
shuffle之前 第一个样本 =1+2+3+466666 66666 hgsg sgs hshshs hsfh shsh sh dh fhh
4+3+2=1                    
shuffle之前 最后一个样本 =h hsr 


ge whreh 
 hgs ghhth
样本数量=10
[{'input_ids': [299, 299, 30917, 30918, 30910, 13, 13, 13, 465, 366, 263, 30919, 30910, 13, 299, 6196, 317, 12578, 398, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [299, 299, 30917, 30918, 30910, 13, 13, 13, 465, 366, 263, 30919, 30910, 13, 299, 6196, 317, 12578, 398, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'input_ids': [10789, 30927, 11408, 271, 9563, 30933, 5282, 266, 310, 317, 30929, 467, 30927, 1063, 1056, 30972, 374, 30965, 30926, 29953, 2065, 30925, 30965,

In [68]:
dataset_eval = get_chained_lm_datasets(lm_datasets,
                        pad_token_id=tokenizer.pad_token_id,
                        global_args_max_length=5,
                                 num_samples=2)
print(dataset_eval)

样本中ids 最大长度 max_len_ids=219
全局训练参数的global_args_max_length=5
cutoff to len = 5
样本数量=2
[{'input_ids': [64790, 64792, 30910, 30939, 31011], 'labels': [64790, 64792, 30910, 30939, 31011]}, {'input_ids': [64790, 64792, 30910, 30939, 31011], 'labels': [64790, 64792, 30910, 30939, 31011]}]


In [33]:
def get_datset_for_pretrain(data_path, tokenizer, block_size=10,global_args_max_length=0,max_samples=0):
    """读取本地包含json/jsonl文件的目录，将目录中所有文件作为dataset，并tokenize，shuffle，返回datasets.dataset"""
    print(123)
    if not (data_path is not None and os.path.exists(data_path)):
        raise ValueError("data_path requires a directory pointing to  .txt files")
    """https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py#L383"""
    data_files_list = glob(f'{data_path}/**/*.txt', recursive=True) 
    logger.info(f"data files: {', '.join(data_files_list)}")
          
    extension = "text"
    # 读取全部txt文档 有N篇文档 那raw_datasets这个list就包含N个篇章级别的文本结果
    #sasmple_by="document" 就是将一个txt当做一个sample全部读取
    raw_datasets = load_dataset(
        extension,
        data_files=data_files_list,
        sample_by="document"
    )
    # 对全部N个文档str 做分词 分词结果也是N个长list 每个list都含有input_ids这个
    tokenized_datasets = raw_datasets.map(
                lambda examples : tokenize_function(examples ,tokenizer),
                batched=True,
                num_proc=2,#data_args.preprocessing_num_workers,
                remove_columns=['text'],#column_names,
                #load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on dataset",
            )
    # N个长list在这一步被按照block_size切成小段 原先是[长1],[长2] 现在仍然为2元素 但是每个里面都是小段[[短1-1],[短1-2]] ,[[短2-1],[短2-2]]
    lm_datasets = tokenized_datasets['train'].map(
                lambda examples :group_texts(examples , block_size),
                batched=False,
                num_proc=2,#data_args.preprocessing_num_workers,
                #load_from_cache_file=not data_args.overwrite_cache,
                #desc=f"Grouping texts in chunks of {block_size}",
                #remove_columns =['attention_mask', 'position_ids',]
            )
    dataset_final = get_chained_lm_datasets(lm_datasets,
                        pad_token_id=tokenizer.pad_token_id,
                        global_args_max_length=global_args_max_length,
                        num_samples=max_samples,
                        tokenizer = tokenizer)
#     logger.info(f"在取样之前 data len ={len(data['train'])}")
#     if max_samples is not None and max_samples > 0:
#             max_samples = min(len(data['train']), max_samples)  # 
#             data['train'] =  data['train'].select(range(max_samples))
#     logger.info(f"在取样之后 data len ={len(data['train'])}")
    
#     column_names = data['train'].column_names  # remove_columns=column_names  ,remove all at once
#     """tokenize_func 中是单样本处理的写法 所以这里的batched只能设置为False"""
#     logger.info("preprocessing dataset...")
#     dataset = data['train'].map(lambda example: tokenize_func(example, tokenizer, global_args),
#                                 batched=False, 
#                                 remove_columns=column_names)
#     dataset = dataset.shuffle(seed=global_args.seed)
#     dataset = dataset.flatten_indices()
    return dataset_final

In [34]:
train_ds = get_datset_for_pretrain(data_path="/kaggle/working/test",
                                  tokenizer =  AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True),
                                  block_size = 10,
                                  global_args_max_length=6,
                                  max_samples=-1)
print(train_ds)

2023-07-31 11:08:57.939 | INFO     | __main__:get_datset_for_pretrain:8 - data files: /kaggle/working/test/test/test0730.txt, /kaggle/working/test/test/test0730_2.txt


123


  0%|          | 0/1 [00:00<?, ?it/s]

Running tokenizer on dataset (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

total_length=219total_length=219

样本中ids 最大长度 max_len_ids=10
全局训练参数的global_args_max_length=6
cutoff to len = 6
样本数量=42
[{'input_ids': [13, 15939, 317, 265, 30927, 5323], 'labels': [13, 15939, 317, 265, 30927, 5323]}, {'input_ids': [30929, 30934, 416, 30929, 30926, 268], 'labels': [30929, 30934, 416, 30929, 30926, 268]}, {'input_ids': [30974, 30916, 822, 30919, 30970, 30925], 'labels': [30974, 30916, 822, 30919, 30970, 30925]}, {'input_ids': [2065, 30925, 30965, 30926, 23347, 268], 'labels': [2065, 30925, 30965, 30926, 23347, 268]}, {'input_ids': [10789, 30927, 11408, 271, 9563, 30933], 'labels': [10789, 30927, 11408, 271, 9563, 30933]}, {'input_ids': [30929, 9563, 30933, 5282, 266, 13], 'labels': [30929, 9563, 30933, 5282, 266, 13]}, {'input_ids': [10038, 5323, 30929, 689, 2277, 30919], 'labels': [10038, 5323, 30929, 689, 2277, 30919]}, {'input_ids': [54804, 35228, 54819, 55720, 55720, 56645], 'labels': [54804, 35228, 54819, 55720, 55720, 56645]}, {'input_ids': [2065, 30925, 30965, 309

In [35]:
eval_ds = get_datset_for_pretrain(data_path="/kaggle/working/test",
                                  tokenizer =  AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True),
                                  block_size = 10,
                                  global_args_max_length=6,
                                  max_samples=5)
print(eval_ds)

2023-07-31 11:09:06.787 | INFO     | __main__:get_datset_for_pretrain:8 - data files: /kaggle/working/test/test/test0730.txt, /kaggle/working/test/test/test0730_2.txt


123


  0%|          | 0/1 [00:00<?, ?it/s]

样本中ids 最大长度 max_len_ids=10
全局训练参数的global_args_max_length=6
cutoff to len = 6
样本数量=5
[{'input_ids': [42660, 55674, 31123, 34380, 39507, 5088], 'labels': [42660, 55674, 31123, 34380, 39507, 5088]}, {'input_ids': [299, 10240, 30919, 438, 1480, 438], 'labels': [299, 10240, 30919, 438, 1480, 438]}, {'input_ids': [10789, 30927, 11408, 271, 9563, 30933], 'labels': [10789, 30927, 11408, 271, 9563, 30933]}, {'input_ids': [30929, 467, 30927, 1063, 1056, 30972], 'labels': [30929, 467, 30927, 1063, 1056, 30972]}, {'input_ids': [13, 30972, 31011, 30966, 31011, 30943], 'labels': [13, 30972, 31011, 30966, 31011, 30943]}]


In [38]:
import math
import os
from dataclasses import dataclass, field
from glob import glob
from itertools import chain
from typing import Optional, List, Dict, Any, Mapping
def fault_tolerance_data_collator(features: List) -> Dict[str, Any]:
    if not isinstance(features[0], Mapping):
        features = [vars(f) for f in features]
    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if type(first["label_ids"][0]) is int else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    try:
        for k, v in first.items():
            if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
                if isinstance(v, torch.Tensor):
                    batch[k] = torch.stack([f[k] for f in features])
                elif isinstance(v, np.ndarray):
                    batch[k] = torch.tensor(np.stack([f[k] for f in features]))
                else:
                    batch[k] = torch.tensor([f[k] for f in features])
    except ValueError:  # quick fix by simply take the first example
        for k, v in first.items():
            if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
                if isinstance(v, torch.Tensor):
                    batch[k] = torch.stack([features[0][k]] * len(features))
                elif isinstance(v, np.ndarray):
                    batch[k] = torch.tensor(np.stack([features[0][k]] * len(features)))
                else:
                    batch[k] = torch.tensor([features[0][k]] * len(features))
    #print("打印fault_tolerance_data_collator处理后的信息")
    #print(f"batch.keys()={batch.keys()},len_batch_key_labels={len(batch['labels'])}    ,len_batch_key_k={len(batch[k])}")
    return batch

In [39]:
fault_tolerance_data_collator(eval_ds)

{'input_ids': tensor([[42660, 55674, 31123, 34380, 39507,  5088],
         [  299, 10240, 30919,   438,  1480,   438],
         [10789, 30927, 11408,   271,  9563, 30933],
         [30929,   467, 30927,  1063,  1056, 30972],
         [   13, 30972, 31011, 30966, 31011, 30943]]),
 'labels': tensor([[42660, 55674, 31123, 34380, 39507,  5088],
         [  299, 10240, 30919,   438,  1480,   438],
         [10789, 30927, 11408,   271,  9563, 30933],
         [30929,   467, 30927,  1063,  1056, 30972],
         [   13, 30972, 31011, 30966, 31011, 30943]])}

In [40]:
class DataCollatorForChatGLM:
    
    # 虽然这个函数中又做了pad和截断 但是实际在pt阶段没有效果 因为之前已经是统一长度而且把尾部短文本用
    # pad token 补齐过了 这里代码留着是为了后面参考改多轮对话用
    def __init__(self,
                 pad_token_id: int,
                 max_length: int = 2048,
                 ignore_label_id: int = -100):
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """根据batch最大长度做padding"""
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)
        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            #padding :虽然预训练之前的步骤做了 长度已经是统一的 但是这个可以继续pad
            #注意 ids和label使用的pad token不同 
            # 在预训练阶段 下面的6行实际没有任何作用 因为pad_len 此时为0 可以打印看看
            pad_len = batch_max_len - len_of_d
            print(f"pad_len in data collator = {pad_len}")
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[: self.max_length]
                label = label[: self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)
        return {'input_ids': input_ids, 'labels': labels}

data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id,
                                           max_length=7)

In [41]:
data_collator(train_ds)

{'input_ids': tensor([[   13, 15939,   317,   265, 30927,  5323],
         [30929, 30934,   416, 30929, 30926,   268],
         [30974, 30916,   822, 30919, 30970, 30925],
         [ 2065, 30925, 30965, 30926, 23347,   268],
         [10789, 30927, 11408,   271,  9563, 30933],
         [30929,  9563, 30933,  5282,   266,    13],
         [10038,  5323, 30929,   689,  2277, 30919],
         [54804, 35228, 54819, 55720, 55720, 56645],
         [ 2065, 30925, 30965, 30926, 23347,   268],
         [30978,   299,  6196, 30927,   268,  6196],
         [30929, 30934,   416, 30929, 30926,   268],
         [  299,   299, 30917, 30918, 30910,    13],
         [30916,  2823, 30921,   317, 30931,   317],
         [16462, 26604, 30919,   299,   416, 30929],
         [30978,   299,  6196, 30927,   268,  6196],
         [30978, 30978, 30978, 30978, 30978, 30910],
         [64790, 64792, 30910, 30939, 31011, 30943],
         [   13, 30972, 31011, 30966, 31011, 30943],
         [  467,   315, 30926,   

In [250]:
# 注意加上*
a=lm_datasets['input_ids']
list(chain(*a))


[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972],
 [30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978],
 [30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917],
 [299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578],
 [13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296],
 [10789, 30927, 11408, 271, 9563, 30933, 5282, 266, 310, 317],
 [30929, 467, 30927, 1063, 1056, 30972, 374, 30965, 30926, 29953],
 [2065, 30925, 30965, 30926, 23347, 268, 30921, 30965, 30916, 30926],
 [5239, 30926, 265, 13, 36545, 31994, 265, 54622, 55282, 31514],
 [42660, 55674, 31123, 34380, 39507, 5088, 14724, 310, 30933, 30920],
 [16462, 26604, 30919, 299, 416, 30929, 30920, 30965, 795, 30933],
 [30916, 2823, 30921, 317, 30931, 317, 30921, 30910, 13, 55058],
 [54804, 35228, 54819, 55720, 55720, 56645, 31123, 31538, 31529, 30995],
 [4571, 30953, 30920, 30933, 13532, 30921, 260, 265, 1298, 265],
 [30974, 30916, 822, 30919, 30970, 30925, 280, 670, 20092,

In [138]:
lm_datasets['train']['input_ids']

[[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972],
  [30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978],
  [30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917],
  [299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578],
  [13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296],
  [10789, 30927, 11408, 271, 9563, 30933, 5282, 266, 310, 317],
  [30929, 467, 30927, 1063, 1056, 30972, 374, 30965, 30926, 29953],
  [2065, 30925, 30965, 30926, 23347, 268, 30921, 30965, 30916, 30926],
  [5239, 30926, 265, 13, 36545, 31994, 265, 54622, 55282, 31514],
  [42660, 55674, 31123, 34380, 39507, 5088, 14724, 310, 30933, 30920],
  [16462, 26604, 30919, 299, 416, 30929, 30920, 30965, 795, 30933],
  [30916, 2823, 30921, 317, 30931, 317, 30921, 30910, 13, 55058],
  [54804, 35228, 54819, 55720, 55720, 56645, 31123, 31538, 31529, 30995],
  [4571, 30953, 30920, 30933, 13532, 30921, 260, 265, 1298, 265],
  [30974, 30916, 822, 30919, 30970, 30925, 2

In [173]:
a=[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972],
  [30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978]]
b=[[30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917],
  [299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578],
  [13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296],]
from itertools import chain
list(chain(a,b))

[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972],
 [30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978],
 [30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917],
 [299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578],
 [13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296]]

In [133]:
# 可以验证
#list(chain(lm_datasets['train']['input_ids'][0] ,lm_datasets['train']['input_ids'][1]))

In [134]:
#可以验证
#tokenized_dataset['train']['input_ids']

In [399]:
a=[1,2]
a[:5]

[1, 2]

In [1]:
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

Cloning into 'chatGLM-6B-QLoRA'...
remote: Enumerating objects: 941, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 941 (delta 123), reused 73 (delta 73), pack-reused 780
Receiving objects: 100% (941/941), 17.26 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (592/592), done.


In [100]:
cd /kaggle/working/chatGLM-6B-QLoRA

/kaggle/working/chatGLM-6B-QLoRA


In [ ]:
!git pull --all --force
#!pip install peft==0.4.0
#!pip install  -U git+https://github.com/huggingface/peft.git
%cd /kaggle/working/chatGLM-6B-QLoRA 
!ls
!pip install -r requirements.txt
#!pip install deepspeed==0.9.5  这个也是需要的 但是目前kaggle 的runtime自带了

In [110]:
ls data/pretrain/xuanyan

2  xy.txt


In [149]:
!git pull --all --force 
#!pip install  -U git+https://github.com/huggingface/peft.git   # 20230717 peft==0.4.0正式发布了 不用调版本了推理完后再训练需要重新升级到0.4.0dev 所以有这句
!deepspeed --include localhost:0,1  pretrain_qlora_chatglm2.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-pt \
  --num_train_samples -1 \
  --num_eval_samples 20 \
  --block_size 1024 \
  --train_data_path ./data/pretrain/xuanyan/ \
  --eval_data_path  ./data/pretrain/xuanyan/ \
  --max_length 1024 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  5e-5 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 682 bytes | 227.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   5d7feba..78dd514  main       -> origin/main
Updating 5d7feba..78dd514
Fast-forward
 pretrain_qlora_chatglm2.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
[2023-07-31 15:42:59,920] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unabl